In [13]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

import json
import pymongo as pm

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [14]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_group_data_for_annotation.csv'
meta = pd.read_csv(path_to_metadata)    

In [16]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append(["leg,armrest,backrest,seat,pedestal,wheel"])        
    if meta.category[i]=="dog":
        parts.append(["eye,mouth,ear,body,tail,neck,leg"])
    if meta.category[i] == "bird":
        parts.append(["beak,wings,feet,tail,body,head"])
    if meta.category[i] == "car":
        parts.append(["wheel,windshield,body,window,door,bumper,lamp"])
meta = meta.assign(parts=pd.Series(parts))

In [17]:
## add iteration name information
_iterationName = 'sketchpad_basic'
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [42]:
## svg string formatting
meta['svg'] = meta['svg'].str.replace('u', '')
meta['svg'] = meta['svg'].str.replace("'",'"') 
meta.svg[2]

'["M34,212c5.3353,-10.6706 6.56215,-22.81075 9,-35c0.26954,-1.34768 -0.33333,-3.66667 1,-4c15.14741,-3.78685 54.94206,4.02897 67,-2c1.19257,-0.59628 0,-2.66667 0,-4c0,-4.23016 1.05755,-27.02877 5,-29c7.58299,-3.7915 118.26093,-13.49271 125,-9c10.97767,7.31845 17,16.31601 17,29c0,0.66667 -0.53333,1.6 0,2c8.79066,6.593 23.15243,5.91456 25,17c1.15474,6.92844 2.44282,28.55718 -3,34c-10.28851,10.28851 -41.02072,5.44214 -54,4c-38.91959,-4.3244 -78.40107,-3.58396 -118,-2c-6.66966,0.26679 -13.34467,0.48805 -20,1c-4.02552,0.30966 -50.12812,6.87188 -53,4c-1.58978,-1.58978 1.32583,-10 2,-10", "M65,217c-8.65513,0 3.04981,29.48318 5,31c5.41249,4.20972 15.93338,7 23,7c22.95419,0 22,-21.0419 22,-37", "M201,218c0,4.23349 -6.08208,13.37689 -3,18c8.88657,13.32985 37.84581,22.30838 48,2c4.12282,-8.24564 -4,-17.83787 -4,-23", "M40,166c9.25689,0 6.43939,51 -11,51c-3.07318,0 5.62563,-3.25126 7,-6c3.11652,-6.23305 7.69274,-12.38549 11,-19c2.58238,-5.16476 2,-12.15409 2,-18c0,-2.53859 -3.97638,-9.34331 -3,-7c

In [6]:
## add empty games list
games = [[] for i in np.arange(len(meta))]
meta = meta.assign(games=pd.Series(games))

In [7]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)

93 unique games worth of data.
2976 unique sketches.


### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [8]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta.js',mode='rU').read())
assert len(J)==len(meta)

In [9]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [10]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_sketchpad_basic']

In [11]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = False
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)
reallyRun = False        

0 of 2976
250 of 2976
500 of 2976
750 of 2976
1000 of 2976
1250 of 2976
1500 of 2976
1750 of 2976
2000 of 2976
2250 of 2976
2500 of 2976
2750 of 2976


In [12]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))

We have 2976 sketches.
We have these kinds: [u'chair', u'car', u'dog', u'bird']
